In [1]:
from Tensor import tensor
import Tensor
from Toolkit import *
import torch

In [2]:
a=tensor([[[2.,2.,3.],[4., 5., 6.]],[[7.,8.,9.],[10.,11.,12.]]])# dimention: (2,2,3)
a.requireGrad(True) 
b=a**-2
c=a**-3
d=b*c
e=d.log() #dimention (2,2,3)
f = e.mean() # scalar
print("grad_e:",e.Grad(a))
print("grad_f:",f.Grad(a)) 

grad_e: [[[-2.5, -2.5, -1.66667], [-1.25, -1.0, -0.83333]], [[-0.71429, -0.625, -0.55556], [-0.5, -0.45455, -0.41667]]]
grad_f: [[[-0.20833, -0.20833, -0.13889], [-0.10417, -0.08333, -0.06944]], [[-0.05952, -0.05208, -0.0463], [-0.04167, -0.03788, -0.03472]]]


In [3]:
print(e.shape)
print(f.shape)
print(a.reshape((4,3)))

(2, 2, 3)
(1,)
[[2.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0], [10.0, 11.0, 12.0]]


In [4]:
#用pytorch对照
a_ = torch.FloatTensor([[[2.,2.,3.],[4., 5., 6.]],[[7.,8.,9.],[10.,11.,12.]]])
a_ = torch.autograd.Variable(a_,requires_grad=True)
b_=a_**-2
c_=a_**-3
d_=b_*c_
e_=torch.log(d_)
_grad = torch.autograd.grad(outputs=e_,inputs=a_,grad_outputs=torch.ones_like(a_))
print("grad_e: ",_grad)
print()
f_ = e_.mean()
f_.backward()
print("grad_f: ",a_.grad.data)

grad_e:  (Variable containing:
(0 ,.,.) = 
 -2.5000 -2.5000 -1.6667
 -1.2500 -1.0000 -0.8333

(1 ,.,.) = 
 -0.7143 -0.6250 -0.5556
 -0.5000 -0.4545 -0.4167
[torch.FloatTensor of size 2x2x3]
,)

grad_f:  
(0 ,.,.) = 
 -0.2083 -0.2083 -0.1389
 -0.1042 -0.0833 -0.0694

(1 ,.,.) = 
 -0.0595 -0.0521 -0.0463
 -0.0417 -0.0379 -0.0347
[torch.FloatTensor of size 2x2x3]



In [5]:
# MSE_vector
label = tensor([[2.1]])
x=tensor([[1.2,2.3,3.4]], require_grad=True)
w=tensor([[0.21],[0.53],[-0.3]] ,require_grad=True)
w.set_compflow(x) # 和x的计算图相关联，由于外积计算有可能造成计算图的id丢失，所以需要先关联起来
b=tensor([0.1] ,require_grad=True)
y = x.dot(w) + b 
# print(y.fullGrad(w)) # 获得全导数矩阵
mse = ((y - label) **2).mean()
print("grad:", mse.fullGrad(w))

grad: [-3.7175988006148257, -7.125397701178416, -10.533196601742006]


In [6]:
label_ = torch.FloatTensor([[2.1]])
label_ = torch.autograd.Variable(label_,requires_grad=True)
x_ = torch.FloatTensor([[1.2,2.3,3.4]])
x_ = torch.autograd.Variable(x_,requires_grad=True)
w_ = torch.FloatTensor([[0.21],[0.53],[-0.3]])
w_ = torch.autograd.Variable(w_,requires_grad=True)
b_ = torch.FloatTensor([0.1])
b_ = torch.autograd.Variable(b_,requires_grad=False)
y_ = torch.mm(x_,w_) + b_
_grad = torch.autograd.grad(outputs=y_,inputs=x_,grad_outputs=torch.ones_like(x_))
# print(_grad)
mse = ((y_ - label_) ** 2).mean()
mse.backward()
print(w_.grad.data)


 -3.7176
 -7.1254
-10.5332
[torch.FloatTensor of size 3x1]



In [7]:
# MSE_matrix
label = tensor([[2.1],[3.1]])
x=tensor([[1.,2.,3.],[4., 5., 6.]], require_grad=True)
w=tensor([[0.21],[0.53],[-0.3]] ,require_grad=True)
w.set_compflow(x) # 和x的计算图相关联，由于外积计算有可能造成计算图的id丢失，所以需要先关联起来
b=tensor([0.1] ,require_grad=True)
y = x.dot(w) + b 
# print(y.fullGrad(w))
mse = ((y - label) **2).mean()
grad_w = mse.fullGrad(w) # 获得全导数张量 ,由于外积计算在trace计算时有问题，不能直接获取偏导数
# print(grad_w)
# 整合全导数
result = [0]*w.shape[0]
a,b = grad_w.shape[1],grad_w.shape[2]
for i in range(0,a):
    for j in range(3):
        result[j] += grad_w[i][i][j+i*(b//2)]
print("mse:", mse)
print("grad result:", result)

mse: [2.1864999999999997]
grad result: [-6.869997493901067, -9.809996491426958, -12.749995488952846]


In [8]:
# pytorch 0.3.0 , 0.4.0以上合并了Tensor和Variable
label_ = torch.FloatTensor([[2.1],[3.1]])
label_ = torch.autograd.Variable(label_,requires_grad=True)
x_ = torch.FloatTensor([[1.,2.,3.],[4., 5., 6.]])
x_ = torch.autograd.Variable(x_,requires_grad=True)
w_ = torch.FloatTensor([[0.21],[0.53],[-0.3]])
w_ = torch.autograd.Variable(w_,requires_grad=True)
b_ = torch.FloatTensor([0.1])
b_ = torch.autograd.Variable(b_,requires_grad=False)
y_ = torch.mm(x_,w_) + b_
_grad = torch.autograd.grad(outputs=y_,inputs=x_,grad_outputs=torch.ones_like(x_))
# print(_grad)
mse = ((y_ - label_) ** 2).mean()
print(mse)
mse.backward()
print(w_.grad.data)

Variable containing:
 2.1865
[torch.FloatTensor of size 1]


 -6.8700
 -9.8100
-12.7500
[torch.FloatTensor of size 3x1]

